# Week3_4 Assignment

## [BASIC](#Basic) 
- Encoder & Decoder Layer 코드를 직접 필사하고 각 함수에 주석을 달 수 있다. 

## [CHALLENGE](#Challenge)
- 텐서의 크기(shape)를 계산할 수 있다. 

## [ADVANCED](#Advanced)
- 완성된 transformer 모델의 모든 학습 가능한 파라미터 이름과 크기(shape)를 출력할 수 있다.

### Informs
이번 과제에서는 "[Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)"의 코드를 필사해본다.   
"Annotated Transformer"는 "Attention is all you need" 논문에서 제안한 transformer 모델을 pytorch 라이브러리로 직접 구현한다.   
코드 필사를 통해 다음을 배울 수 있다.    
- Encoder, Decoder 구조
- Attention Mechanism
- "residual connection", "layer normalization" 등의 구조 

코드 필사를 시작하기 앞서, transformer 모델의 최종 구조를 살펴보자.    

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_full.png?raw=true" width="500" align="center"/>

최종 모델은 `EncoderDecoder()` 클래스에 여러 인스턴스를 생성자의 입력 파라미터로 넣어 생성한다.    
앞으로 우리는 `EncoderDecoder()` 클래스와 같은 여러 클래스들을 구현하고 연결할 것이다. 따라서 대략적인 클래스간의 관계를 살펴보고 이해한다면 보다 큰 그림을 가지고 코드 필사를 할 수 있을 것이다. 

Transformer 모델은 크게 4가지 클래스로 구현된다.    
- Frame
    - frame 역할을 하는 `EncoderDecoder` 클래스
- Input Embedding & Encoding
    - 입력값을 벡터화하는 `Embeddings`, `PositionalEncoding`
- Encoder & Decoder
    - 각 6개 layer를 갖고 있는 `Encoder`, `Decoder`
    - layer 1층을 구현한 `EncoderLayer`, `DecoderLayer`
- Sublayer
    - `EncoderLayer`, `DecoderLayer` 내부에서 사용되는 Sublayer 클래스인 `MultiHeadAttiontion`, `PositionwiseFeedForward`
    - Sublayer 클래스들을 연결하는 `SublayerConnection`
    
아래 좌측 도식에서 각 클래스의 색상은 아래 우측 도식(transformer 구조)의 색상과 맵핑되어 있다.    
각 클래스의 역할과 클래스 간 연결 관계를 생각하면서 transformer를 코드로 구현해보자.   


<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_map.png?raw=true" width="400" height="400" align="left"/>
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_transformer.png?raw=true" width="300" height="400" align="right"/>



In [2]:
import os
import sys
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

## Basic

### Frame
- `EncoderDecoder`

아래 도식은 `EncoderDecoder` 클래스의 `forward()`, `encode()`, `decode()` 메소드를 도식화 한 것이다.    
 
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_encoderdecoder.png?raw=true" width=500>


- `Generator`

In [3]:
## encoderdecoder frame 
class EncoderDecoder(nn.Module):
    
    # src: sequence to the encoder, tgt: sequence to the decoder, src_embed: embedded src, tgt_embed: embedded tgt
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
    
    # src_mask: additive mask for the src, tgt_mask: additive mask for the src
    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    # encode
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    # memory(인코딩된 output)= model.encode(src, src_mask)
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [4]:
## final output(linear + softmax)
# output size :  (input size x vocab size) -> 마지막 차원에 대한 argmax -> input size    
class Generator(nn.Module):  

    # d_model: size of input, vocab: size of output  
    def __init__(self, d_model, vocab):
      super(Generator, self).__init__()
      self.proj = nn.Linear(d_model, vocab)    
    
    # F.log_softmax = F.softmax() + torch.log()(softmax 결과에 log 취한 값) 
    # dim.... 
    def forward(self, x):
      return F.log_softmax(self.proj(x), dim = -1)

### Encoder
- `Encoder`
- `EncoderLayer`
- `SublayerConnection`
- Reference
    - Layer Normalization
        - [한국어 설명](https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/)
        - [torch official docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
    - Residual Connection
        - [한국어 설명](https://itrepo.tistory.com/36)
    - pytorch ModuleList
        - [torch official docs](https://pytorch.org/docs/1.9.1/generated/torch.nn.ModuleList.html)


In [5]:
# nn.ModeuleList(): nn.Module을 리스트로 정리하는 법, 각 레이어를 리스트에 전달하고 레이어의 iterator를 만들어 forward를 간단히 할 수 있음
# nn.Module(): NN(neural network) 모듈의 기본 클래스
# nn.Sequential(): input이 하나일 때(즉 각 layer를 데이터가 순차적으로 지나갈 때 사용)


In [6]:
# N개의 layer 만들기(N=6)
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [7]:
# stack of N layers
class Encoder(nn.Module):

    def __init__(self, layer, N):
      super(Encoder, self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm(layer.size)
    
    
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


In [8]:
# LayerNorm: 각 input의 feature들에 대한 평균과 분산을 구해 batch의 각 input을 normalize(batch size와 관련 x)
# (cf) BatchNorm: 각 feature들의 평균과 분산을 구해 batch의 각 feature를 normalize
class LayerNorm(nn.Module):
    
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    # layer norm을 위해 batch 각 샘플에 대한 mean, std 계산 후 정규화
    def forward(self, x):
        mean = x.mean(-1, keepdim = True)
        std = x.std(-1, keepdim = True)
        return self.a_2 * (x-mean) / (std + self.eps) + self.b_2

In [9]:
# residual connection : 이전 layer의 output을 현재 layer의 output에 추가하는 것  
class SublayerConnection(nn.Module):
    
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

In [10]:
# 단일 encoderlayer : multiheadattention > feed_forward 형태
class EncoderLayer(nn.Module):
    
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size
        
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x,x,x, mask))
        return self.sublayer[1](x, self.feed_forward)

### Decoder
- `Decoder`
- `DecoderLayer`

In [11]:
class Decoder(nn.Module):
    
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [12]:
class DecoderLayer(nn.Module):

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size 
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

### Sublayer
- `attention` 함수

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_attention.png?raw=true" width="500" align="center"/>  

- `MultiHeadedAttention`
- `PositionwiseFeedForward`

### Challenge


### Q1. 위 도식에 따라 `score`, `p_attn`, `attention` 을 구하라 

In [13]:
## scaled dot product attention
# query: input의 embedding, key/value: target
def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    #scores: 각 단어상호간 가중치 표현 테이블, sqrt통해 가중치 편차 줄어들도록 함 
    scores = torch.matmul(query, key.transpose(-2,-1))/math.sqrt(d_k)

    # masking 한 부분이 -1e9(매우 작은 값)으로 변경
    if mask is not None:
        scores = scores.masked_fill(mask == 0 , -1e9)
    # softmax 통해 가중치가 확률로 변환(masking부분 = 0)
    p_attn = F.softmax(scores, dim=-1)

    if dropout is not None:
        p_attn = dropout(p_attn)
    
    # attn확률 * value    
    return torch.matmul(p_attn, value), p_attn


###Q2. query, key, value가 모두 (m, d_k) shape의 matrix라고 가정할 때, `score`, `p_attn`, `attention`의 shape을 각각 구하라
- score : m*m
- p_attn : m*m
- attention : m*d_k

In [14]:
q = torch.rand(size = (12,8,1,64))
k = torch.rand(size = (12,8,1,64))
v = torch.rand(size = (12,8,1,64))
d_k = q.size(-1)
d_k #64
scores = torch.matmul(q, k.transpose(-2,-1))/math.sqrt(d_k)
scores.size()
p_attn = F.softmax(scores, dim=-1)
at = torch.matmul(p_attn, v)
at.size()

torch.Size([12, 8, 1, 64])

### (아래의 **Q3을 먼저 풀고 돌아오세요**) Q4.  query, key, value가 모두 (12, 8, 1, 64) shape의 tensor라고 가정할 때 , `score`, `p_attn`, `attention`의 shape을 각각 구하라

- score : (12,8,1,1)
- p_attn : (12,8,1,1)
- attention : (12,8,1,64)

- `MultiHeadedAttention`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_multihead.png?raw=true" width="300" align="center"/>  

In [15]:
## multihead attention : 다중 패턴 및 표현 학습(cf. attention: 하나의 표현 학습)
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0

        # d_k(차원을 축소시켜서 병렬 attention 수행) = d_model/num of head
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p = dropout)
    
    
    def forward(self, query, key, value, mask=None):

        if mask is not None:
            mask = mask.unsqueeze(1) #dim =1에 차원을 하나 추가하라(a, b) -> (a,1,b)
        nbatches = query.size(0)

        # d_model 에서 batch에 linear projection --> h*d_k
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1,2) for l, x in zip(self.linears, (query, key, value))]

        # batch에 projecte된 모든 vector에 대해 attention 적용
        x, self.attn = attention(query, key, value, mask = mask, dropout = self.dropout)

        # concat
        x = x.transpose(1,2).contiguous().view(nbatches, -1, self.h* self.d_k)
        return self.linears[-1](x)


            

In [16]:
q = torch.rand(12,512)
k = torch.rand(12,512)
v = torch.rand(12,512)
nn.Linear(512, 512)(q).view(q.size(0), -1, 8, 64).transpose(1,2).size()

torch.Size([12, 8, 1, 64])

### Q3.  query, key, value가 모두 (12, 512) shape의 matrix이고, h 값이 8 이라고 가정할 때, 아래 값의 shape을 각각 구하라

- `d_k` (d_k = d_model // h) : 512/8 = 64
- `nn.Linear(d_model, d_model)(query)` : (12,512)
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k)` : (12,1,8,64)
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2)` : (12,8,1,64)

- `PositionwiseFeedForward`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pwff.png?raw=true" width="300" align="center"/>  

In [17]:
## fully connected feed forward network
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout = 0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    # relu = max(0,x)     
    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

### Input Embedding & Encoding
- `Embeddings`
    - [pytorch official docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [18]:
class Embeddings(nn.Module):

    ## nn.Embedding(#vocab_size, #embedding_vector_size), d_model = 512
    def __init__(self, d_model, vocab):
      super(Embeddings, self).__init__()
      self.lut = nn.Embedding(vocab, d_model)
      self.d_model = d_model
    
    # normalization
    def forward(self, x):
      return self.lut(x) * math.sqrt(self.d_model)

- `PositionalEncoding`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pe.png?raw=true" width="500" align="center"/>  

- `position` 변수 설명
    - 모든 position (=최대 토큰 개수)의 값을 갖고 있는 matrix
- `div_term` 변수 설명

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_div.png?raw=true" width="500" align="center"/>  
- `Embedding` + `Encoding` 도식화 

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_emb_enc.png?raw=true" width="400" align="center"/>  


In [19]:
class PositionalEncoding(nn.Module):

    # max_len: max sequence length
    def __init__(self, d_model, dropout, max_len = 5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p = dropout)

        ## compute the positional encodings in log space(why log? numerical overflow 방지)
        # pe: input matrix에 더하기 위해 input matrix와 같은 size로 만들기
        pe = torch.zeros(max_len, d_model)
        # 1차원 -> 2차원으로 unsqueeze
        position = torch.arange(0,max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0)/d_model))

        # sin: 짝수 차원, cos: 홀수차원
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        #register_buffer : optimizer 없데이트 하지 않고, state_dict() 확인가능, GPU연산 가능
        self.register_buffer('pe',pe)

        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)],requires_grad = False)
        return self.dropout(x)

In [20]:
pos = torch.arange(0,512)
pos.unsqueeze(1)
pos.unsqueeze(1).size()
#position = torch.arange(0,512).unsqueeze(1)
#torch.exp(torch.arange(0, 512, 2)).size()  #0-512까지 2step >> 256개 
div_term = torch.exp(torch.arange(0, 512, 2) * -(math.log(10000.0)/512))
div_term.size()
#torch.tensor(position * div_term).size()

torch.Size([256])

### Q4.  max_len이 512이고, d_model이 512라고 가정할 때, `position`과 `div_term`의 shape을 구하라

- `position` : (512,1)
- `div_term` : (256)
- `position * div_term` : (512, 256)

### Advanced

### Finally Build Model
- Xavier Initialization
    - [한국어 자료](https://huangdi.tistory.com/8)
    - [pytorch official docs](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_)

In [21]:
# model construct!
def make_model(src_vocab, tgt_vocab, 
               N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
                           Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff) ,dropout), N),
                           nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
                           nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
                           Generator(d_model, tgt_vocab))
    
    # initialize parameters
    for p in model.parameters():
        if p.dim() >1 :
            nn.init.xavier_uniform_(p)
    return model
            

In [22]:
model = make_model(10,10)

### Q5. 위 코드로 만든 모델의 모든 파라미터의 이름과 크기 (shape) 을 출력하라

In [23]:
print(model)

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): Linear(in_features=512, out_features=512, bias=True)
            (2): Linear(in_features=512, out_features=512, bias=True)
            (3): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()